<h4>Data Quality Checks</h4>

In this notebook we are trying to uncover any issues in the dataset by doing the following checks:
<ul>
<li><b>Completeness:</b> The dataset is not fully complete. Various columns have missing values and two columns are entirely empty.</li>
<li><b>Consistency:</b> The dataset is inconsistent, with some files having extra columns not present in others.</li>



<li><b>Uniqueness:</b> The dataset is not unique, certain attributes having different names but covering the same or similar measurements, like Radio Altitude Pilot and Altitude (AGL)</li>

</ul>

<b>Input:</b>
Raw dataset, as received from the flight simulator.

<b>Output:</b>
Findings to help address different issues when pre-processing the dataset in data_pre_progressing_physics_model notebook

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

directory = '/Users/amahfouz/Downloads/csv_data/mixed'

# Get a list of all CSV files in the directory
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

def remove_dtypes_and_scenario_lines(df):
    df.drop(0, inplace=True)
    df.drop(1, inplace=True)
    df.drop(df.index[-1], inplace=True)
    empty_columns = df.columns[df.isnull().all()].tolist()
    df.drop(columns=empty_columns, inplace=True)

In [ ]:
for file in csv_files:
    file_path = os.path.join(directory, file)
    # Read each CSV file excluding the header row
    df = pd.read_csv(file_path)
    # Concatenate the data to the main DataFrame
    # Read the CSV file
    df.drop(0, inplace=True)
    df.drop(1, inplace=True)
    empty_columns = df.columns[df.isnull().all()].tolist()
    nan_columns = df.columns[df.isnull().any()].tolist()
    print("Columns with NaN values:", nan_columns)
    print("Empty Columns: ",empty_columns)

In [ ]:
# Dictionary to store column data types across files
column_data_types = {}

# Loop through each file and extract column data types
for file in csv_files:
    file_path = os.path.join(directory, file)
    # Read CSV file into a DataFrame
    df = pd.read_csv(file_path)
    remove_dtypes_and_scenario_lines(df)

    # Extract column data types and store them in the dictionary
    column_data_types[file] = df.dtypes

# Compare data types for each column across files
for column in column_data_types[csv_files[0]].index:
    unique_data_types = set(data_types[column] for data_types in column_data_types.values())

    # If there is more than one unique data type for a column, print the column name and different data types
    if len(unique_data_types) > 1:
        print(f"Column '{column}' has different data types: {unique_data_types}")
    else:
        print(f"Column '{column}' has consistent data type: {unique_data_types.pop()}")


In [120]:
# Loop through each file and extract column data types
def find_data_types(df):
    # Compare data types for each column across files
    column_data_types = {}
    for column in df.columns:
        # Additional checks based on column content (modify as per your domain knowledge)
        sample_values = df[column].dropna().astype(str).sample(5, replace=True)  # Take a sample of non-null values
        # Check if the column contains numerical data
        if pd.to_numeric(sample_values, errors='coerce').notna().all():
            column_data_types[column] = 'numerical'

        # Check if the column contains date data (assuming dates are in a specific format)
        elif pd.to_datetime(sample_values, errors='coerce').notna().all():
            column_data_types[column] = 'date'

        # Check if the column contains categorical data (a limited set of unique values)
        elif len(sample_values.unique()) < len(sample_values):
            column_data_types[column] = 'categorical'

        # If none of the above conditions are met, it's likely free-text and treated as an object
        else:
            column_data_types[column] = 'freetext'

    return column_data_types

file_column_data_types = {}
for file in csv_files:
    file_path = os.path.join(directory, file)
    # Read CSV file into a DataFrame
    df = pd.read_csv(file_path)
    remove_dtypes_and_scenario_lines(df)
    file_column_data_types[file] = find_data_types(df)

count=0
for file in csv_files:
    for column in file_column_data_types[file].keys():
        unique_data_types = set()
        for file_name, data_types in file_column_data_types.items():
            try:
                unique_data_types.add(data_types[column])
                # If there is more than one unique data type for a column, print the column name and different data types
            except:
                count= count+1
                print(f"\033[0;31m Column '{column}' in '{file}' not found in file '{file_name}'")
        if len(unique_data_types) > 1:
            print(f"\033[0;31m Column '{column}' has different data types: {unique_data_types}")
            count= count+1
if count == 0:
    print(f"\033[0;32m All good!")

/var/folders/23/thv06wgd00j6tfbg1sr_s4mc0000gn/T/ipykernel_43245/515333592.py:32: DtypeWarning: Columns (0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,142,143,144,145,146,147,148,149,150,151,152,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/23/thv06wgd00j6tfbg1sr_s4mc0000gn/T/ipykernel_43245/515333592.py:15: UserWarning: Could not infer format, so each e

 Column 'Flight ID' in 'SimData_2023.06.01_13.21.58.csv' not found in file 'SimData_2022.10.20_16.07.20.csv'
 Column 'Flight ID' in 'SimData_2023.06.01_13.21.58.csv' not found in file 'SimData_2022.07.28_15.13.02.csv'
 Column 'GPS 1 NAV ID' in 'SimData_2023.06.01_13.21.58.csv' not found in file 'SimData_2022.10.20_16.07.20.csv'
 Column 'GPS 1 NAV ID' in 'SimData_2023.06.01_13.21.58.csv' not found in file 'SimData_2022.07.28_15.13.02.csv'
 Column 'GPS 1 DME Time' in 'SimData_2023.06.01_13.21.58.csv' not found in file 'SimData_2022.10.20_16.07.20.csv'
 Column 'GPS 1 DME Time' in 'SimData_2023.06.01_13.21.58.csv' not found in file 'SimData_2022.07.28_15.13.02.csv'
 Column 'GPS 1 DME Distance' in 'SimData_2023.06.01_13.21.58.csv' not found in file 'SimData_2022.10.20_16.07.20.csv'
 Column 'GPS 1 DME Distance' in 'SimData_2023.06.01_13.21.58.csv' not found in file 'SimData_2022.07.28_15.13.02.csv'
 Column 'GPS 1 DME Speed' in 'SimData_2023.06.01_13.21.58.csv' not found in file 'SimData_2022

/var/folders/23/thv06wgd00j6tfbg1sr_s4mc0000gn/T/ipykernel_43245/515333592.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  elif pd.to_datetime(sample_values, errors='coerce').notna().all():
/var/folders/23/thv06wgd00j6tfbg1sr_s4mc0000gn/T/ipykernel_43245/515333592.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  elif pd.to_datetime(sample_values, errors='coerce').notna().all():
/var/folders/23/thv06wgd00j6tfbg1sr_s4mc0000gn/T/ipykernel_43245/515333592.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  elif pd.to_datetime(sample_values, errors='coerce').notna().al